In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Build a Module

In [74]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,6,3)
        self.conv2 = nn.Conv2d(6,16,3)
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)

        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)

        x = x.view(-1, self.num_flat_features(x))

        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [75]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [76]:
input_images = torch.rand(1, 1, 32, 32)
output = net.forward(input_images)

In [78]:
output.shape

torch.Size([1, 576])

## Use Sequential and Module to build LeNet5

In [ ]:
import torch.nn.ConvNe

In [93]:
class LeNet5(nn.Module):
    def __init__(self, num_classes) -> None:
        super(LeNet5, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc = nn.Linear(400,120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [94]:
lenet5 = LeNet5(10)
print(lenet5)

LeNet5(
  (layer1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=400, out_features=120, bias=True)
  (relu): ReLU()
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)


In [95]:
input_images = torch.rand(16,1,32,32)
output = lenet5(input_images)

In [100]:
print(output)

tensor([[ 0.1324,  0.0213, -0.2384, -0.1363,  0.3165, -0.1074,  0.0943, -0.0893,
         -0.0378, -0.1162],
        [ 0.0823,  0.0124, -0.1083, -0.1358,  0.0847, -0.0305,  0.0192,  0.0822,
          0.0955, -0.1411],
        [ 0.1380, -0.0438, -0.0674, -0.1749,  0.2370,  0.0078,  0.0849,  0.1142,
          0.0426, -0.0698],
        [ 0.1743,  0.0533, -0.1069, -0.2342,  0.1609, -0.0586,  0.0518,  0.0472,
          0.0549, -0.0705],
        [-0.0032,  0.1048, -0.1678,  0.0024,  0.0633, -0.0056,  0.1120,  0.0723,
          0.0945, -0.0153],
        [ 0.0823,  0.0209, -0.1536, -0.0438,  0.1681,  0.0258,  0.0870,  0.0914,
          0.0316, -0.0524],
        [ 0.0531,  0.1440, -0.0773, -0.0504,  0.2889, -0.0094,  0.0963,  0.1119,
          0.0293, -0.0982],
        [ 0.0689,  0.0850, -0.0166, -0.0169,  0.2404, -0.0090,  0.0196,  0.0191,
          0.0359,  0.0030],
        [ 0.0753,  0.0429, -0.2578, -0.1282,  0.2292, -0.0980,  0.0788,  0.0568,
          0.1542, -0.0310],
        [ 0.0329,  

## Handwritten digit classification

In [101]:
import torchvision
import torchvision.transforms as transforms

# Define relevant libraries, and alias where appropriate
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 10

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transforms_compose = transforms.Compose([
                    transforms.Resize((32,32)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean = (0.1307,), std = (0.3081,))
                    ])


# Load dataset and preprocessing 
train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    transform=transforms_compose,
    download=True
)

test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    transform=transforms_compose,
    download=True
)

train_loader = torch.utils.data.DataLoader(
    dataset = train_dataset,
    batch_size=batch_size,
    shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True
)



In [106]:
train_loader.dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data/MNIST
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

# Hyper parameters 


In [114]:

model = LeNet5(num_classes).to(device) # Use model in CPU/GPU

# Setting loss function
loss_fn = nn.CrossEntropyLoss()

# # Setting optimizer with the model parameters and learning rate
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate
)

# This is defines to print how many steps are remaining when training 
total_step = len(train_loader)
print(total_step)

938


In [121]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.to(device)
        labels = labels.to(device)
        
        #Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        	
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        		
        if (i+1) % 400 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
        		           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [400/938], Loss: 0.1071
Epoch [1/10], Step [800/938], Loss: 0.0986
Epoch [2/10], Step [400/938], Loss: 0.1126
Epoch [2/10], Step [800/938], Loss: 0.0504
Epoch [3/10], Step [400/938], Loss: 0.0649
Epoch [3/10], Step [800/938], Loss: 0.0294
Epoch [4/10], Step [400/938], Loss: 0.0049
Epoch [4/10], Step [800/938], Loss: 0.0765
Epoch [5/10], Step [400/938], Loss: 0.0079
Epoch [5/10], Step [800/938], Loss: 0.0128
Epoch [6/10], Step [400/938], Loss: 0.0024
Epoch [6/10], Step [800/938], Loss: 0.0039
Epoch [7/10], Step [400/938], Loss: 0.0234
Epoch [7/10], Step [800/938], Loss: 0.0005
Epoch [8/10], Step [400/938], Loss: 0.0351
Epoch [8/10], Step [800/938], Loss: 0.0027
Epoch [9/10], Step [400/938], Loss: 0.0004
Epoch [9/10], Step [800/938], Loss: 0.0869
Epoch [10/10], Step [400/938], Loss: 0.0074
Epoch [10/10], Step [800/938], Loss: 0.0004


In [122]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
  
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {:.2f} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 98.87 %


In [125]:
state = {
        'epoch': epoch,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
}
torch.save(state, './data/MNIST/LeNet5.pth')